In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
season2018 = pd.read_csv("/content/drive/My Drive/MySaudiLeagueData2018.csv")
season2019 = pd.read_csv("/content/drive/My Drive/MySaudiLeagueData2019.csv")

In [ ]:
season1819.drop(columns=["home_coach", "away_coach"], inplace=True)
season1920.drop(columns=["home_coach", "away_coach"], inplace=True)

## initial data exploration


In [ ]:
season1819.head()

,Unnamed: 0,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
0,0,Al Ettifaq,Al Raed,1,1,4-2-3-1,4-2-3-1,Ummer Al-Senin,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",2018/2019
1,1,Al Hazem,Al Wehda,0,0,4-2-3-1,4-2-3-1,Fagner Daponte,"['malik asselah', 'fagner daponte', 'khaled al...",2018/2019
2,2,Ohod Club,Al Nassr,1,2,4-3-2-1,4-3-2-1,Luis Dialisson de Souza Alves,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",2018/2019
3,3,Qadisiyah,Al Fateh,0,0,4-3-3,4-2-3-1,Yassen Barnawi,"['jack duncan', 'mohammed khubrani', 'adnan fa...",2018/2019
4,4,Al Hilal,Al Fayha,1,0,4-1-3-2,3-5-2,Mohammed Alburayk,"['ali al-habsi', 'botia', 'yasser al shahrani'...",2018/2019


In [ ]:
season1819.shape

(240, 10)

We have 240 matches which makes snse, since we have 16 teams, and 30 round, so we got 16*30 / 2 = 240

In [ ]:
season1819.isnull().sum().sum()

0

In [ ]:
season1920.head()

,Unnamed: 0,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
0,0,Al Nassr,Damac,2,0,4-2-3-1,4-2-3-1,Matthew Conger,"['brad jones', 'maicon pereira roque', 'abdulr...",2019/2020
1,1,Alahli,Al Adalh,1,1,4-2-3-1,4-2-3-1,Bakary Papa Gassama,"['mohammed al-owais', 'mohamed fatel', 'hussei...",2019/2020
2,2,Al Hilal,Abha,4,2,4-2-3-1,4-2-3-1,Dennis Higler,"['abdullah al maiouf', 'jang hyun-soo', 'hassa...",2019/2020
3,3,Al Taawoun,Al Hazem,1,1,4-2-3-1,4-2-3-1,Luis Godinho,"['cassio dos anjos', 'ricardo machado', 'nildo...",2019/2020
4,4,Al Ittihad,Al Raed,3,1,4-4-2,4-2-3-1,Svein Oddvar Moen,"['fawaz al qarni', 'ziad al sahefi', 'hamdan a...",2019/2020


In [ ]:
season1920.shape

(184, 10)

As of this moment 184 matches has been played which means that the dataset is complete

In [ ]:
season1920.isnull().sum().sum()

0

In [ ]:
season1819["home"].unique()

array(['Al Ettifaq', 'Al Hazem', 'Ohod Club', 'Qadisiyah', 'Al Hilal',
       'Al Shabab', 'Al Faisaly', 'Alahli', 'Al Fateh', 'Al Nassr',
       'Al Fayha', 'Al Raed', 'Al Ittihad', 'Al Taawoun', 'Al Wehda',
       'Al-Batin'], dtype=object)

In [ ]:
season1920["home"].unique()

array(['Al Nassr', 'Alahli', 'Al Hilal', 'Al Taawoun', 'Al Ittihad',
       'Al Fateh', 'Al Faisaly', 'Al Wehda', 'Al Adalh', 'Al Fayha',
       'Al Shabab', 'Al Ettifaq', 'Al Raed', 'Damac', 'Abha', 'Al Hazem'],
      dtype=object)

In [ ]:
len(season1819["home"].unique())

16

In [ ]:
len(season1920["home"].unique())

16

In [ ]:
season1819.columns

Index(['Unnamed: 0', 'home', 'away', 'home_goals', 'away_goals', 'home_format',
       'away_format', 'referee', 'players', 'season'],
      dtype='object')

In [ ]:
season1920.columns

Index(['Unnamed: 0', 'home', 'away', 'home_goals', 'away_goals', 'home_format',
       'away_format', 'referee', 'players', 'season'],
      dtype='object')

## Little discussion

I am not sure if this is the optimal format for the data, I am worried that since entries are split between Home & Away, when we want to perform some eda or predict the outcome we would have 2 target variables to work against. 
Another thing is that the machine would probably consider being home or away as significantly different when it is not.
Maybe we should change the format so that we have the following 
* [Team, Current points, IsHome, opponent, opponent current points, Date, Season, referee, Goals Scored]

Now the target variable is goals scored.
we can try to predict the final outcome of the match by running the model twice on both teams, then by comparing the goals we give the final result as Win, defeat, draw. 

## Date cleaning & processing, before getting back to EDA

In [ ]:
# before we do that we should join the two dataframes together
df = pd.concat([season1819, season1920])

In [ ]:
df.head()

,Unnamed: 0,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
0,0,Al Ettifaq,Al Raed,1,1,4-2-3-1,4-2-3-1,Ummer Al-Senin,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",2018/2019
1,1,Al Hazem,Al Wehda,0,0,4-2-3-1,4-2-3-1,Fagner Daponte,"['malik asselah', 'fagner daponte', 'khaled al...",2018/2019
2,2,Ohod Club,Al Nassr,1,2,4-3-2-1,4-3-2-1,Luis Dialisson de Souza Alves,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",2018/2019
3,3,Qadisiyah,Al Fateh,0,0,4-3-3,4-2-3-1,Yassen Barnawi,"['jack duncan', 'mohammed khubrani', 'adnan fa...",2018/2019
4,4,Al Hilal,Al Fayha,1,0,4-1-3-2,3-5-2,Mohammed Alburayk,"['ali al-habsi', 'botia', 'yasser al shahrani'...",2018/2019


In [ ]:
df.tail()

,Unnamed: 0,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
179,179,Al Faisaly,Al Taawoun,2,1,4-2-3-1,4-2-3-1,Faisal Al Balwi,"['mostafa malayka', 'igor rossi branco', 'moha...",2019/2020
180,180,Alahli,Al Hazem,4,2,4-2-3-1,5-4-1,Yasser Al Sultan,"['mohammed al-owais', 'mohammed khubrani', 'lu...",2019/2020
181,181,Al Wehda,Al Shabab,0,3,4-2-3-1,4-3-3,Yasser Al Sultan,"['abdullah al-jdani', 'hassan al-tambakti', 'h...",2019/2020
182,182,Al Adalh,Al Ettifaq,0,1,4-1-4-1,4-2-3-1,Georgi Kabakov,"['ali al mazyadi', 'ce as mandaw', 'abdulla al...",2019/2020
183,183,Al Nassr,Al Hilal,1,4,4-1-4-1,4-2-3-1,Majid Al Shamrani,"['brad jones', 'abdulelah ali alamri', 'abdulr...",2019/2020


In [ ]:
df.shape

(424, 10)

In [ ]:
df.loc[0]

,Unnamed: 0,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
0,0,Al Ettifaq,Al Raed,1,1,4-2-3-1,4-2-3-1,Ummer Al-Senin,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",2018/2019
0,0,Al Nassr,Damac,2,0,4-2-3-1,4-2-3-1,Matthew Conger,"['brad jones', 'maicon pereira roque', 'abdulr...",2019/2020


In [ ]:
#the index seems to be mixed between the two seasons

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
#let's get rid of the Unnamed: 0 variable
df.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df.columns

Index(['home', 'away', 'home_goals', 'away_goals', 'home_format',
       'away_format', 'referee', 'players', 'season'],
      dtype='object')

In [ ]:
df.head()

,home,away,home_goals,away_goals,home_format,away_format,referee,players,season
0,Al Ettifaq,Al Raed,1,1,4-2-3-1,4-2-3-1,Ummer Al-Senin,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",2018/2019
1,Al Hazem,Al Wehda,0,0,4-2-3-1,4-2-3-1,Fagner Daponte,"['malik asselah', 'fagner daponte', 'khaled al...",2018/2019
2,Ohod Club,Al Nassr,1,2,4-3-2-1,4-3-2-1,Luis Dialisson de Souza Alves,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",2018/2019
3,Qadisiyah,Al Fateh,0,0,4-3-3,4-2-3-1,Yassen Barnawi,"['jack duncan', 'mohammed khubrani', 'adnan fa...",2018/2019
4,Al Hilal,Al Fayha,1,0,4-1-3-2,3-5-2,Mohammed Alburayk,"['ali al-habsi', 'botia', 'yasser al shahrani'...",2018/2019


##### Now we will change the format of the data


In [ ]:
# since every entry now is based on home I'll create a feature called IsHome and set it to 1

df["IsHome"] = 1

I will space out rows by changing the index to only even numbers, then inject the opposing team between the rows in the odd index

In [ ]:
len([x for x in range(0, len(df)*2, 2)])

424

In [ ]:
len(df)

424

In [ ]:
df.index = [x for x in range(0, len(df) * 2, 2)]

In [ ]:
df.columns

Index(['home', 'away', 'home_goals', 'away_goals', 'home_format',
       'away_format', 'referee', 'players', 'season', 'IsHome'],
      dtype='object')

In [ ]:
# I'll change the order of the columns so that it is close to the desired format

In [ ]:
df = df[["home", "home_format", "IsHome", "away", "away_format", "players","referee", "season", "home_goals", "away_goals"]]

In [ ]:
df.columns


Index(['home', 'home_format', 'IsHome', 'away', 'away_format', 'players',
       'referee', 'season', 'home_goals', 'away_goals'],
      dtype='object')

In [ ]:
df.head()

,home,home_format,IsHome,away,away_format,players,referee,season,home_goals,away_goals
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,1
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1,2
6,Qadisiyah,4-3-3,1,Al Fateh,4-2-3-1,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0,0
8,Al Hilal,4-1-3-2,1,Al Fayha,3-5-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,1,0


In [ ]:
for i in [x for x in range(1, len(df) * 2 + 1, 2)]:
    old_entry = df.loc[i-1]
    df.loc[i] = [old_entry.away, old_entry.away_format, 0, old_entry.home, old_entry.home_format, old_entry.players, old_entry.referee, old_entry.season, old_entry.away_goals, np.nan]
    

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.head(10)

,home,home_format,IsHome,away,away_format,players,referee,season,home_goals,away_goals
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1.0,1.0
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1.0,NaN
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0.0,0.0
3,Al Wehda,4-2-3-1,0,Al Hazem,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0.0,NaN
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1.0,2.0
5,Al Nassr,4-3-2-1,0,Ohod Club,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,2.0,NaN
6,Qadisiyah,4-3-3,1,Al Fateh,4-2-3-1,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0.0,0.0
7,Al Fateh,4-2-3-1,0,Qadisiyah,4-3-3,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0.0,NaN
8,Al Hilal,4-1-3-2,1,Al Fayha,3-5-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,1.0,0.0
9,Al Fayha,3-5-2,0,Al Hilal,4-1-3-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,0.0,NaN


In [ ]:
# Now we will change the columns so that it aligns with our format

df.drop("away_goals", inplace=True, axis=1)

df.columns = ["Team1", "format", "IsHome", "Opponent", "Opponent_format", "Players", "Referee", "Season", "GoalsScored"]

In [ ]:
df.head(10)

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1.0
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1.0
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0.0
3,Al Wehda,4-2-3-1,0,Al Hazem,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0.0
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1.0
5,Al Nassr,4-3-2-1,0,Ohod Club,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,2.0
6,Qadisiyah,4-3-3,1,Al Fateh,4-2-3-1,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0.0
7,Al Fateh,4-2-3-1,0,Qadisiyah,4-3-3,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0.0
8,Al Hilal,4-1-3-2,1,Al Fayha,3-5-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,1.0
9,Al Fayha,3-5-2,0,Al Hilal,4-1-3-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,0.0


In [ ]:
df.GoalsScored = df.GoalsScored.astype("int64")

In [ ]:
df.head(2)

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1


#### Adding Current Points feature & final game state

In [ ]:
df.Team1.unique()

array(['Al Ettifaq', 'Al Raed', 'Al Hazem', 'Al Wehda', 'Ohod Club',
       'Al Nassr', 'Qadisiyah', 'Al Fateh', 'Al Hilal', 'Al Fayha',
       'Al Shabab', 'Al Ittihad', 'Al Faisaly', 'Al-Batin', 'Alahli',
       'Al Taawoun', 'Damac', 'Al Adalh', 'Abha'], dtype=object)

In [ ]:
df["pts"] = 0
df["opponent_pts"] = 0
df["result"] = 0

In [ ]:

for season in ["2018/2019", "2019/2020"]:
    
    season_df = df.loc[(df["Season"] == season )]

    points = {'Al Ettifaq':0,
              'Al Raed':0,
              'Al Hazem':0,
               'Al Wehda':0,
               'Ohod Club':0,
                'Al Nassr':0,
               'Qadisiyah':0,
               'Al Fateh':0,
               'Al Hilal':0,
               'Al Fayha':0,
                'Al Shabab':0,
                'Al Ittihad':0,
                'Al Faisaly':0,
                'Al-Batin':0,
                'Alahli':0,
               'Al Taawoun':0,
                'Damac':0,
                'Al Adalh':0,
                'Abha':0}

    for i in season_df.index:
        if i % 2 == 0 :
            Team1 = season_df.loc[i, "Team1"]
            Team2 = season_df.loc[i, "Opponent"]


            season_df.loc[i,"pts"] = points[Team1]
            season_df.loc[i, "opponent_pts"] = points[Team1]

            season_df.loc[i+1,"pts"] = points[Team2]
            season_df.loc[i+1, "opponent_pts"] = points[Team1]


            if season_df.loc[i, "GoalsScored"] > season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 3
                season_df.loc[i, "result"] = "Win"
                season_df.loc[i+1, "result"] = "Defeat"
            
            elif season_df.loc[i, "GoalsScored"] == season_df.loc[i+1, "GoalsScored"]:
                points[Team1] += 1
                points[Team2] += 1
                season_df.loc[i, "result"] = "Draw"
                season_df.loc[i+1, "result"] = "Draw"

            else:
                points[Team2] += 3
                season_df.loc[i, "result"] = "Defeat"
                season_df.loc[i+1, "result"] = "Win"

        df.loc[df["Season"] == season] = season_df




/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df.head(50)

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored,pts,opponent_pts,result
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw
3,Al Wehda,4-2-3-1,0,Al Hazem,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1,0,0,Defeat
5,Al Nassr,4-3-2-1,0,Ohod Club,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,2,0,0,Win
6,Qadisiyah,4-3-3,1,Al Fateh,4-2-3-1,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0,0,0,Draw
7,Al Fateh,4-2-3-1,0,Qadisiyah,4-3-3,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0,0,0,Draw
8,Al Hilal,4-1-3-2,1,Al Fayha,3-5-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,1,0,0,Win
9,Al Fayha,3-5-2,0,Al Hilal,4-1-3-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,0,0,0,Defeat


In [ ]:
season_df.tail(50)

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored,pts,opponent_pts,result
798,Al Ittihad,4-3-3,1,Al Shabab,4-1-4-1,"['marcelo grohe', 'bruno uvini', 'mansour al h...",Bjorn Kuipers,2019/2020,5,19,19,Win
799,Al Shabab,4-1-4-1,0,Al Ittihad,4-3-3,"['marcelo grohe', 'bruno uvini', 'mansour al h...",Bjorn Kuipers,2019/2020,1,28,19,Defeat
800,Al Hazem,3-4-3,1,Al Ittihad,4-3-3,"['malik asselah', 'abdullah haif al shammari',...",Bobby Madden,2019/2020,1,22,22,Draw
801,Al Ittihad,4-3-3,0,Al Hazem,3-4-3,"['malik asselah', 'abdullah haif al shammari',...",Bobby Madden,2019/2020,1,22,22,Draw
802,Al Shabab,4-2-3-1,1,Al Adalh,4-1-4-1,"['farouk ben mustapha', 'mohammed salem', 'abd...",Majid Al Shamrani,2019/2020,4,28,28,Win
803,Al Adalh,4-1-4-1,0,Al Shabab,4-2-3-1,"['farouk ben mustapha', 'mohammed salem', 'abd...",Majid Al Shamrani,2019/2020,0,14,28,Defeat
804,Al Raed,4-2-3-1,1,Al Fateh,4-1-4-1,"['azzedine doukha', 'ezequiel palomeque', 'moh...",Matej Jug,2019/2020,0,31,31,Draw
805,Al Fateh,4-1-4-1,0,Al Raed,4-2-3-1,"['azzedine doukha', 'ezequiel palomeque', 'moh...",Matej Jug,2019/2020,0,18,31,Draw
806,Al Wehda,4-2-3-1,1,Damac,4-2-3-1,"['abdullah al-jdani', 'botia', 'hamad al jizan...",Faisal Al Balwi,2019/2020,3,33,33,Win
807,Damac,4-2-3-1,0,Al Wehda,4-2-3-1,"['abdullah al-jdani', 'botia', 'hamad al jizan...",Faisal Al Balwi,2019/2020,2,17,33,Defeat


Adding the round feature so we can visualize over rounds

In [ ]:
df["Round"] = 0

for season in ["2018/2019", "2019/2020"]:

    season_df = df.loc[df["Season"] == season]
    teams = season_df.Team1.unique()
    
    for team in teams:
        df_team = season_df.loc[season_df["Team1"] == team]

        df_team.Round = range(1, len(df_team)+1)

        season_df.loc[season_df["Team1"] == team] = df_team
    
    df.loc[df["Season"] == season] = season_df
        

df.Round = df.Round.astype("int64")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored,pts,opponent_pts,result,Round
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw,1
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw,1
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw,1
3,Al Wehda,4-2-3-1,0,Al Hazem,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw,1
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1,0,0,Defeat,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,Al Shabab,4-3-3,0,Al Wehda,4-2-3-1,"['abdullah al-jdani', 'hassan al-tambakti', 'h...",Yasser Al Sultan,2019/2020,3,31,39,Win,23
844,Al Adalh,4-1-4-1,1,Al Ettifaq,4-2-3-1,"['ali al mazyadi', 'ce as mandaw', 'abdulla al...",Georgi Kabakov,2019/2020,0,17,17,Defeat,23
845,Al Ettifaq,4-2-3-1,0,Al Adalh,4-1-4-1,"['ali al mazyadi', 'ce as mandaw', 'abdulla al...",Georgi Kabakov,2019/2020,1,29,17,Win,23
846,Al Nassr,4-1-4-1,1,Al Hilal,4-2-3-1,"['brad jones', 'abdulelah ali alamri', 'abdulr...",Majid Al Shamrani,2019/2020,1,45,45,Defeat,23


In [ ]:
not_matching_rounds_index = []
for x in range(0, len(df), 2):
    if df.loc[x, "Round"] != df.loc[x+1, "Round"]:
        not_matching_rounds_index.append(x)
        not_matching_rounds_index.append(x+1)

In [ ]:
df.loc[not_matching_rounds_index]

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored,pts,opponent_pts,result,Round
90,Al Ettifaq,4-2-3-1,1,Alahli,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Igor Pajac,2018/2019,6,8,8,Win,5
91,Alahli,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Igor Pajac,2018/2019,2,13,8,Defeat,6
92,Al Shabab,4-2-3-1,1,Al Hilal,4-1-4-1,"['farouk ben mustapha', 'valerica gaman', 'art...",Hessel Steegstra,2018/2019,0,9,9,Defeat,6
93,Al Hilal,4-1-4-1,0,Al Shabab,4-2-3-1,"['farouk ben mustapha', 'valerica gaman', 'art...",Hessel Steegstra,2018/2019,1,12,9,Win,5
96,Al Ettifaq,4-2-3-1,1,Al Hazem,4-2-3-1,"['rais mbolhi', 'ahmed al habeb', 'hussein el ...",Rais Mbolhi,2018/2019,1,11,11,Win,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,Alahli,4-2-3-1,0,Al Hilal,4-2-3-1,"['abdullah al maiouf', 'mohammed jahfali', 'ya...",Szymon Marciniak,2019/2020,1,23,24,Defeat,13
696,Al Faisaly,4-2-3-1,1,Al Raed,4-2-3-1,"['mostafa malayka', 'aqeel baalghyth al sahbi'...",Mohamed Al Heweish,2019/2020,0,20,20,Defeat,13
697,Al Raed,4-2-3-1,0,Al Faisaly,4-2-3-1,"['mostafa malayka', 'aqeel baalghyth al sahbi'...",Mohamed Al Heweish,2019/2020,1,23,20,Win,14
698,Al Hilal,4-2-3-1,1,Al Wehda,4-2-3-1,"['abdullah al maiouf', 'abdulla al hafez', 'ya...",Ovidiu Hategan,2019/2020,3,27,27,Win,13


In [ ]:
goal_keepers_2018 = {'Al Ettifaq', 'Al Raed', 'Al Hazem', 'Al Wehda', 'Ohod Club',
       'Al Nassr', 'Qadisiyah', 'Al Fateh', 'Al Hilal', 'Al Fayha',
       'Al Shabab', 'Al Ittihad', 'Al Faisaly', 'Al-Batin', 'Alahli',
       'Al Taawoun'}


goal_keepers_2020 = {'Al Nassr', 'Damac', 'Alahli', 'Al Adalh', 'Al Hilal':'abdullah al maiouf', 'Abha',
       'Al Taawoun', 'Al Hazem', 'Al Ittihad', 'Al Raed', 'Al Fateh',
       'Al Shabab', 'Al Faisaly', 'Al Fayha', 'Al Wehda', 'Al Ettifaq'}


In [ ]:
df.head(10)

,Team1,format,IsHome,Opponent,Opponent_format,Players,Referee,Season,GoalsScored,pts,opponent_pts,result,Round
0,Al Ettifaq,4-2-3-1,1,Al Raed,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw,1
1,Al Raed,4-2-3-1,0,Al Ettifaq,4-2-3-1,"['rais mbolhi', 'saeed al rabiei', 'hussein el...",Ummer Al-Senin,2018/2019,1,0,0,Draw,1
2,Al Hazem,4-2-3-1,1,Al Wehda,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw,1
3,Al Wehda,4-2-3-1,0,Al Hazem,4-2-3-1,"['malik asselah', 'fagner daponte', 'khaled al...",Fagner Daponte,2018/2019,0,0,0,Draw,1
4,Ohod Club,4-3-2-1,1,Al Nassr,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,1,0,0,Defeat,1
5,Al Nassr,4-3-2-1,0,Ohod Club,4-3-2-1,"['zouheir laaroubi', 'enes sipovic', 'hussein ...",Luis Dialisson de Souza Alves,2018/2019,2,0,0,Win,1
6,Qadisiyah,4-3-3,1,Al Fateh,4-2-3-1,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0,0,0,Draw,1
7,Al Fateh,4-2-3-1,0,Qadisiyah,4-3-3,"['jack duncan', 'mohammed khubrani', 'adnan fa...",Yassen Barnawi,2018/2019,0,0,0,Draw,1
8,Al Hilal,4-1-3-2,1,Al Fayha,3-5-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,1,0,0,Win,1
9,Al Fayha,3-5-2,0,Al Hilal,4-1-3-2,"['ali al-habsi', 'botia', 'yasser al shahrani'...",Mohammed Alburayk,2018/2019,0,0,0,Defeat,1
